In [1]:
# Import necessary dependencies
import pandas as pd
from sqlalchemy import create_engine
from sklearn import metrics
import pickle
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

In [2]:
URL = "dbbikes.cvwut6jnqsvn.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes"
USER = "admin"
PASSWORD = "password"

In [3]:
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{URL}:{PORT}/{DB}",echo = True)



# Weather Data Preparation
___

In [4]:
# create dataframe of weatherData
query = "SELECT * FROM dbbikes.Weather;"
dfWeather = pd.read_sql_query(query, engine)
dfWeather["last_update"] = pd.to_datetime(dfWeather.last_update) #change last_update feature to datetime type

2023-04-16 16:11:54,798 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-16 16:11:54,800 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-16 16:11:54,981 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-16 16:11:54,981 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-16 16:11:55,073 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-16 16:11:55,074 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-16 16:11:55,256 INFO sqlalchemy.engine.Engine SELECT * FROM dbbikes.Weather;
2023-04-16 16:11:55,257 INFO sqlalchemy.engine.Engine [raw sql] {}


In [5]:
dfWeather.tail()

,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,weather_base,main_temp,feels_like,main_temp_min,...,clouds_all,last_update,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,city_id,city_name,cod
3011,-6.2672,53.344,802,Clouds,scattered clouds,03d,stations,290.77,290.57,290.31,...,40,2023-04-16 15:50:00,2,2031847,IE,2023-04-16 06:23:58,2023-04-16 20:25:30,2964574,Dublin,200
3012,-6.2672,53.344,802,Clouds,scattered clouds,03d,stations,290.77,290.57,290.31,...,40,2023-04-16 15:50:00,2,2031847,IE,2023-04-16 06:23:58,2023-04-16 20:25:30,2964574,Dublin,200
3013,-6.2672,53.344,802,Clouds,scattered clouds,03d,stations,290.77,290.57,290.31,...,40,2023-04-16 15:50:00,2,2031847,IE,2023-04-16 06:23:58,2023-04-16 20:25:30,2964574,Dublin,200
3014,-6.2672,53.344,802,Clouds,scattered clouds,03d,stations,290.77,290.57,290.31,...,40,2023-04-16 16:00:03,2,2031847,IE,2023-04-16 06:23:58,2023-04-16 20:25:30,2964574,Dublin,200
3015,-6.2672,53.344,802,Clouds,scattered clouds,03d,stations,289.82,289.4,289.82,...,40,2023-04-16 16:09:29,2,2037117,IE,2023-04-16 06:23:58,2023-04-16 20:25:30,2964574,Dublin,200


In [6]:
# Select categorical columns
categorical_columns = dfWeather[['weather_main', 'weather_description', 'weather_icon', 'weather_base']].columns

# Convert data type to category for these columns
for column in categorical_columns:
    dfWeather[column] = dfWeather[column].astype('object')

In [7]:
# Select columns that should be floats
float_columns = dfWeather[['coord_lon', 'coord_lat', 'main_temp', 'feels_like',"main_temp_min","wind_speed",
                    "main_temp_max",]].columns
# Convert data type to floats
for column in float_columns:
    dfWeather[column] = dfWeather[column].astype('float64')

# Select data types that should be integers
int_columns=dfWeather[['weather_id', 'main_pressure', 'main_humidity', 'main_visibility',"clouds_all",]].columns

for column in int_columns:
    dfWeather[column] = dfWeather[column].astype('int64')


In [8]:
# Select only important Weather features
dfWeather = dfWeather[["weather_description","main_temp","main_humidity",'wind_speed',"last_update"]]

In [9]:
dfWeather.sort_values('last_update', inplace = True)

# Bikes Data Preparation
___

In [10]:
query = "SELECT * FROM dbbikes.Bike;"
dfBikes = pd.read_sql_query(query,engine)
dfBikes["last_update"] = pd.to_datetime(dfBikes.last_update)
# Select continuous values and change to appropriate type
dfBikes.sort_values('last_update', inplace = True)

2023-04-16 16:11:56,069 INFO sqlalchemy.engine.Engine SELECT * FROM dbbikes.Bike;
2023-04-16 16:11:56,071 INFO sqlalchemy.engine.Engine [raw sql] {}


In [11]:
dfBikes.dtypes

number                            int64
contract_name                    object
name                             object
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
status                           object
last_update              datetime64[ns]
dtype: object

#Merging the two Dataframes
___

In [12]:
dfMerged = pd.merge_asof(dfBikes,dfWeather,on="last_update",direction="nearest")

In [13]:
df = dfMerged.drop(["contract_name","name","status"],axis=1)

In [14]:
df['month'] = df['last_update'].dt.month
df['dayofweek'] = df['last_update'].dt.dayofweek
df['hour'] = df['last_update'].dt.hour
df['minute'] = df['last_update'].dt.minute

In [15]:
df.dtypes


number                            int64
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
weather_description              object
main_temp                       float64
main_humidity                     int64
wind_speed                      float64
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [16]:
df.tail(1000)


,number,bike_stands,available_bike_stands,available_bikes,last_update,weather_description,main_temp,main_humidity,wind_speed,month,dayofweek,hour,minute
226273,50,40,14,26,2023-04-16 15:04:31,light rain,290.42,73,4.12,4,6,15,4
226274,27,20,9,11,2023-04-16 15:04:33,light rain,290.42,73,4.12,4,6,15,4
226275,108,35,26,9,2023-04-16 15:04:34,light rain,290.42,73,4.12,4,6,15,4
226276,101,30,27,3,2023-04-16 15:04:34,light rain,290.42,73,4.12,4,6,15,4
226277,107,40,40,0,2023-04-16 15:04:38,light rain,290.42,73,4.12,4,6,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227268,82,22,16,6,2023-04-16 16:09:39,scattered clouds,289.82,71,4.63,4,6,16,9
227269,26,20,12,8,2023-04-16 16:09:43,scattered clouds,289.82,71,4.63,4,6,16,9
227270,9,24,0,24,2023-04-16 16:09:49,scattered clouds,289.82,71,4.63,4,6,16,9
227271,86,38,22,16,2023-04-16 16:09:51,scattered clouds,289.82,71,4.63,4,6,16,9


In [17]:
df.to_csv("mergedData.csv")
df.dtypes

number                            int64
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
weather_description              object
main_temp                       float64
main_humidity                     int64
wind_speed                      float64
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

# Training Data Model
___

In [18]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()


In [19]:
# Transform weather description to integers
df['weather_description'] = label_encoder.fit_transform(df['weather_description'])

In [20]:
df.dtypes


number                            int64
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
weather_description               int32
main_temp                       float64
main_humidity                     int64
wind_speed                      float64
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [21]:
df = df.drop("last_update",1)

C:\Users\Bilo\AppData\Local\Temp\ipykernel_16792\3447826205.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("last_update",1)


In [22]:
input_features = ['number','hour','minute','month','weather_description','main_temp', 'main_humidity', 'wind_speed','dayofweek']

In [23]:
input = df[input_features]
target = df[["available_bikes",'dayofweek']]

In [24]:
X = input
Y = target.available_bikes
X_train, X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
regressor = RandomForestRegressor(n_estimators=15)
regressor.fit(X_train, Y_train)

# Write to a pickle file
with open('bike_predictor.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)


y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})

print('Test Set Predictions for Station: ')
print(df)
print()

#Accuracy score
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(Y_test, y_pred))


Test Set Predictions for Station: 
        Actual  Predicted
6621         3   4.066667
216695       0   2.800000
145959      16  15.266667
137233       4   6.066667
162104       3   4.000000
...        ...        ...
216722       1   5.200000
52295       40  40.000000
32570       17  16.800000
140953      14  13.666667
128354      15  12.866667

[68182 rows x 2 columns]

Mean Absolute Error: 1.34552703478505
Mean Squared Error: 5.7490744186651765
Root Mean Squared Error: 2.397722756839326
The accuracy score :  0.9325023076137768


# Testing PKL file
___

In [25]:
row = [0]*9

In [26]:
x_testing = pd.DataFrame([row],columns=input_features)


In [27]:
inputData = {'number':12,'hour': 10,'minute':30,'month': 4, 'weather_description': 3, 'main_temp': 279.60, 'main_humidity': 1025, 'wind_speed': 6.6, 'dayofweek': 3}

In [28]:
def dataframe(df, dict):
    df['number'][0]=dict['number']
    df['hour'][0] = dict['hour']
    df['minute'][0] = dict['minute']
    df['month'][0] = dict['month']
    df['weather_description'][0] = dict['weather_description']
    df['main_temp'][0] = dict['main_temp']
    df['main_humidity'][0] = dict['main_humidity']
    df['wind_speed'][0] = dict['wind_speed']
    df['dayofweek'][0] = dict['dayofweek']

In [29]:
dataframe(X,inputData)

In [30]:
with open('bike_predictor.pkl', 'rb') as handle:
    regressor = pickle.load(handle)

prediction = regressor.predict(X)
prediction[0]


3.1333333333333333